In [ ]:
import pandas as pd
from gensim.models import word2vec
import nltk
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 
from gensim.parsing.preprocessing import remove_stopwords, preprocess_string
from collections import Counter
import string

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
df = pd.read_csv('amazon_reviews_us_Electronics_v1_00.tsv',sep='\t',error_bad_lines=False )

b'Skipping line 9076: expected 15 fields, saw 22\nSkipping line 19256: expected 15 fields, saw 22\nSkipping line 24313: expected 15 fields, saw 22\nSkipping line 47211: expected 15 fields, saw 22\nSkipping line 54295: expected 15 fields, saw 22\nSkipping line 56641: expected 15 fields, saw 22\nSkipping line 63067: expected 15 fields, saw 22\n'
b'Skipping line 93796: expected 15 fields, saw 22\n'
b'Skipping line 132806: expected 15 fields, saw 22\nSkipping line 164631: expected 15 fields, saw 22\nSkipping line 167019: expected 15 fields, saw 22\nSkipping line 167212: expected 15 fields, saw 22\n'
b'Skipping line 198103: expected 15 fields, saw 22\nSkipping line 199191: expected 15 fields, saw 22\nSkipping line 202841: expected 15 fields, saw 22\nSkipping line 218228: expected 15 fields, saw 22\nSkipping line 235900: expected 15 fields, saw 22\n'
b'Skipping line 277761: expected 15 fields, saw 22\nSkipping line 304582: expected 15 fields, saw 22\nSkipping line 312029: expected 15 fields,

In [ ]:
file = open("corpus.txt", "a",encoding="utf-8") 

for index, row in df.iterrows():
    file.write(remove_stopwords(str(row['review_body']))+'\n')
file.close()

In [ ]:
sentences = []
with open('corpus.txt', 'r',encoding="utf-8") as f:
    sentences = f.readlines()
# sentences = sentences[:1_000_000]

In [ ]:
def modified_tokenize(sentence):
  sentence = list(map(lambda x: x.lower(), word_tokenize(sentence)))
  for i in range(len(sentence)):
    if string.punctuation in sentence[i]:
      if sentence[i][-1] in string.punctuation:
        sentence[i] = sentence[:-1]
      else:
        sentence.remove(i)
  return sentence

In [ ]:
word_count = Counter()
sentences_in_words = []
for i in range(len(sentences)):
    sentences_in_words.append(modified_tokenize(sentences[i]))
    
    for word in sentences_in_words[i]:
        
        if len(word)>1 and type(word)==str and string.punctuation not in word:
          word_count.update({str(word): 1})
        

In [ ]:
model = word2vec.Word2Vec(sentences_in_words)

In [ ]:
model.wv.most_similar(['best'], topn=8)

[('greatest', 0.7494461536407471),
 ('worst', 0.7168004512786865),
 ('nicest', 0.6873034238815308),
 ('better', 0.6098830699920654),
 ('finest', 0.6093013882637024),
 ('clearest', 0.6038334965705872),
 ('superior', 0.6003611087799072),
 ('cleanest', 0.585466742515564)]

In [ ]:
model.save('basic_electronics_review.model')
model.wv.save_word2vec_format('basic_electronics_review.model.bin', binary=True)
# model = word2vec.Word2Vec.load_word2vec_format('/tmp/vectors.bin', binary=True)

In [ ]:
word_count.most_common(20)

[('br', 362634),
 ('it', 294907),
 ('great', 276061),
 ('the', 227954),
 ("n't", 227447),
 ('sound', 218407),
 ('good', 192688),
 ("'s", 154428),
 ('works', 142218),
 ('quality', 127830),
 ('use', 118798),
 ('this', 115486),
 ('product', 108743),
 ('like', 104893),
 ('price', 100791),
 ('work', 100161),
 ('headphones', 96374),
 ('34', 92014),
 ('speaker', 88745),
 ('tv', 85063)]

In [ ]:
common_words = ['great','sound','good','works','quality','product','price','like','work','heaphones','tv']

In [ ]:
for word in common_words:
  print(word,model.wv.most_similar([word], topn=3))

great [('fantastic', 0.8480622172355652), ('awesome', 0.8367749452590942), ('good', 0.8209725022315979)]
sound [('sounds', 0.7705094814300537), ('sounding', 0.6725955009460449), ('imaging', 0.6256886124610901)]
good [('decent', 0.8965661525726318), ('great', 0.8209725022315979), ('excellent', 0.7313956618309021)]
works [('worked', 0.8000662326812744), ('performs', 0.6535431146621704), ('functioned', 0.6287904977798462)]
quality [('fidelity', 0.7584156394004822), ('quality..', 0.6597788333892822), ('quality.', 0.6188468933105469)]
product [('item', 0.8698714375495911), ('products', 0.6288233995437622), ('seller', 0.5384914875030518)]
price [('prices', 0.6787002086639404), ('pricing', 0.6664294004440308), ('cost', 0.6651983261108398)]
like [('silly', 0.5628789663314819), ('prefer', 0.5460392236709595), ('weird', 0.5392311811447144)]
work [('worked', 0.7315524816513062), ('works', 0.6161193251609802), ('working', 0.6011919379234314)]
heaphones [('in-ears', 0.732327938079834), ('headphones

In [ ]:
print("Raw sentence:",df['review_body'][101])
print("Stop words removed sentence:",sentences[101])

Raw sentence: First off, I bought this product on sale for $25 off of Amazon. What a great deal, this bluetooth speaker is LOUD! It's got fair bass levels for the size and price point. Both front and back of the device emit sound, though front is louder. I think this gives it better quality and more range. I use this out by the pool and plan on using for fishing trips and tailgates. The only reason i did not give it 5 stars, when pressing the volume +/- the music stops for a &#34;boop&#34; noise. Also, when powering on, changing modes, or pairing a device, the speaker plays a womens voice that sounds almost comical. It's a little goofy, but at this price point who cares. This speaker is comparable to high end units of the same size, with just slightly less bass.<br /><br />Thanks Amazon!
Stop words removed sentence: First off, I bought product sale $25 Amazon. What great deal, bluetooth speaker LOUD! It's got fair bass levels size price point. Both device emit sound, louder. I think gi